In [1]:
import pandas as pd
import os
import json
import yaml
import glob
# import rethinkdb as r
from collections import defaultdict
from qtools import Submitter
from encode import manifest_helpers as m

from tqdm import tnrange, tqdm_notebook
pd.set_option("display.max_colwidth", 10000)

In [2]:
density_runner = '/home/bay001/projects/codebase/rbp-maps/maps/plot_map.py'
annotation_dir = '/projects/ps-yeolab3/bay001/maps/current_annotations/se/'
control_annotation_dir = '/projects/ps-yeolab3/bay001/maps/current_annotations/erics_controls/nonredundant_renamed/se/'

params = {
    'peak':{
        'output_dir' : '/projects/ps-yeolab3/bay001/maps/current_annotations/se_peak_bigbeds/',
        'clip_manifest' : '/projects/ps-yeolab3/bay001/reference_data/misc_ENCODE/ENCODE_FINAL_ANNOTATIONS.uidsonly.txt.manifest.txt',
        'prefix' : 'peak',
        'peak_dir' : '/home/elvannostrand/data/clip/CLIPseq_analysis/ENCODE_FINALforpapers_20180205/hg19/'
    },
    'peak_blacklistremoved':{
        'output_dir' : '/projects/ps-yeolab3/bay001/maps/current_annotations/se_peak_blacklist_removed_bigbeds/',
        'clip_manifest' : '/projects/ps-yeolab3/bay001/reference_data/misc_ENCODE/ENCODE_FINAL_ANNOTATIONS.uidsonly.txt.manifest.txt',
        'prefix' : 'peak',
        'peak_dir' : '/home/elvannostrand/data/clip/CLIPseq_analysis/ENCODE_FINALforpapers_20180205/hg19/'
    },
    'idr':{
        'output_dir' : '/projects/ps-yeolab3/bay001/maps/current_annotations/se_peak_idr_bigbeds/',
        'clip_manifest' : '/projects/ps-yeolab3/bay001/reference_data/misc_ENCODE/ENCODE_FINAL_ANNOTATIONS.uidsonly.txt.manifest.txt',
        'prefix' : 'peak',
        'peak_dir' : '/home/elvannostrand/data/clip/CLIPseq_analysis/ENCODE_FINALforpapers_20180205/hg19/IDR/'
    },
}

In [3]:
current_date = '7-1-2018'
bash_scripts_dir = '/projects/ps-yeolab3/bay001/maps/bash_scripts/{}'.format(current_date)
! mkdir $bash_scripts_dir

mkdir: cannot create directory `/projects/ps-yeolab3/bay001/maps/bash_scripts/7-1-2018': File exists


In [4]:
inputnormed_names = ['chrom','start','end','l10p','l2fc','strand']

def filter_and_change_namefield(fn, out_file, p, fc):
    """
    In order for a bed file to be accepted into the bedToBigBed script, 
    compressed.bed files must have proper names (string) in field 4 and 
    proper score (0) in field 5. This function filters and modifies this.
    """
    df = pd.read_table(fn, names=inputnormed_names)
    df = df[(df['l10p']>=p) & (df['l2fc']>=fc)]
    df['l10p'] = 'peak'
    df['l2fc'] = 0
    df.to_csv(out_file, sep='\t', index=False, header=False)
    
# all_peaks = glob.glob(os.path.join(params['peak_blacklistremoved']['peak_dir'],'*.compressed.bed'))

# Iterate over IDR peaks and convert to bigbed

In [ ]:
p = 0
fc = 0

all_peaks = glob.glob(os.path.join(params['idr']['peak_dir'],'*.out.0102merged.bed.blacklist_removed.bed'))
print(len(all_peaks))

chrom_sizes = '/projects/ps-yeolab/genomes/hg19/hg19.chrom.sizes'
progress = tnrange(len(all_peaks))

for peak in all_peaks:
    out_file = os.path.join(params['idr']['output_dir'], os.path.basename(peak) + '.p{}f{}.bed'.format(p, fc))
    sorted_bed_file = out_file + '.sorted.bed'
    bb_file = sorted_bed_file + '.bb'
    
    filter_and_change_namefield(peak, out_file, p, fc)
    ! sort -k1,1 -k2,2n $out_file > $sorted_bed_file
    ! bedToBigBed $sorted_bed_file $chrom_sizes $bb_file
    progress.update(1)

# Iterate over peakfiles and convert to bigbed

In [5]:
p = 3

all_peaks = glob.glob(os.path.join(params['peak']['peak_dir'],'*.compressed.bed'))
print(len(all_peaks))

chrom_sizes = '/projects/ps-yeolab/genomes/hg19/hg19.chrom.sizes'
progress = tnrange(len(all_peaks))

for fc in [1, 2]:
    for peak in all_peaks:
        out_file = os.path.join(params['peak']['output_dir'], os.path.basename(peak) + '.p{}f{}.bed'.format(p, fc))
        sorted_bed_file = out_file + '.sorted.bed'
        bb_file = sorted_bed_file + '.bb'

        filter_and_change_namefield(peak, out_file, p, fc)
        ! sort -k1,1 -k2,2n $out_file > $sorted_bed_file
        ! bedToBigBed $sorted_bed_file $chrom_sizes $bb_file
        progress.update(1)

446


HBox(children=(IntProgress(value=0, max=446), HTML(value=u'')))

pass1 - making usageList (24 chroms): 27 millis
pass2 - checking and writing primary data (11106 records, 6 fields): 35 millis
pass1 - making usageList (25 chroms): 3 millis
pass2 - checking and writing primary data (3624 records, 6 fields): 34 millis
pass1 - making usageList (25 chroms): 8 millis
pass2 - checking and writing primary data (19526 records, 6 fields): 75 millis
pass1 - making usageList (24 chroms): 10 millis
pass2 - checking and writing primary data (18557 records, 6 fields): 77 millis
pass1 - making usageList (24 chroms): 3 millis
pass2 - checking and writing primary data (7296 records, 6 fields): 22 millis
pass1 - making usageList (24 chroms): 2 millis
pass2 - checking and writing primary data (1316 records, 6 fields): 8 millis
pass1 - making usageList (24 chroms): 3 millis
pass2 - checking and writing primary data (9438 records, 6 fields): 17 millis
pass1 - making usageList (24 chroms): 4 millis
pass2 - checking and writing primary data (4944 records, 6 fields): 18 mil

pass1 - making usageList (24 chroms): 1 millis
pass2 - checking and writing primary data (541 records, 6 fields): 6 millis
pass1 - making usageList (25 chroms): 3 millis
pass2 - checking and writing primary data (11531 records, 6 fields): 35 millis
pass1 - making usageList (23 chroms): 6 millis
pass2 - checking and writing primary data (10145 records, 6 fields): 39 millis
pass1 - making usageList (25 chroms): 2 millis
pass2 - checking and writing primary data (4141 records, 6 fields): 12 millis
pass1 - making usageList (24 chroms): 7 millis
pass2 - checking and writing primary data (13031 records, 6 fields): 46 millis
pass1 - making usageList (24 chroms): 2 millis
pass2 - checking and writing primary data (1378 records, 6 fields): 9 millis
pass1 - making usageList (24 chroms): 3 millis
pass2 - checking and writing primary data (6211 records, 6 fields): 13 millis
pass1 - making usageList (25 chroms): 31 millis
pass2 - checking and writing primary data (62541 records, 6 fields): 121 mill

pass1 - making usageList (25 chroms): 5 millis
pass2 - checking and writing primary data (10168 records, 6 fields): 30 millis
pass1 - making usageList (25 chroms): 3 millis
pass2 - checking and writing primary data (5664 records, 6 fields): 15 millis
pass1 - making usageList (24 chroms): 1 millis
pass2 - checking and writing primary data (1632 records, 6 fields): 5 millis
pass1 - making usageList (24 chroms): 3 millis
pass2 - checking and writing primary data (9308 records, 6 fields): 16 millis
pass1 - making usageList (25 chroms): 3 millis
pass2 - checking and writing primary data (4797 records, 6 fields): 18 millis
pass1 - making usageList (25 chroms): 5 millis
pass2 - checking and writing primary data (8617 records, 6 fields): 28 millis
pass1 - making usageList (25 chroms): 5 millis
pass2 - checking and writing primary data (6919 records, 6 fields): 23 millis
pass1 - making usageList (25 chroms): 8 millis
pass2 - checking and writing primary data (14185 records, 6 fields): 49 millis

...

pass1 - making usageList (23 chroms): 4 millis
pass2 - checking and writing primary data (17212 records, 6 fields): 49 millis
pass1 - making usageList (25 chroms): 5 millis
pass2 - checking and writing primary data (8827 records, 6 fields): 30 millis
pass1 - making usageList (24 chroms): 7 millis
pass2 - checking and writing primary data (12209 records, 6 fields): 45 millis
pass1 - making usageList (23 chroms): 3 millis
pass2 - checking and writing primary data (3521 records, 6 fields): 14 millis
pass1 - making usageList (25 chroms): 3 millis
pass2 - checking and writing primary data (9938 records, 6 fields): 22 millis
pass1 - making usageList (25 chroms): 1 millis
pass2 - checking and writing primary data (1384 records, 6 fields): 4 millis
pass1 - making usageList (24 chroms): 1 millis
pass2 - checking and writing primary data (562 records, 6 fields): 5 millis
pass1 - making usageList (24 chroms): 2 millis
pass2 - checking and writing primary data (7706 records, 6 fields): 19 millis
p

pass1 - making usageList (25 chroms): 4 millis
pass2 - checking and writing primary data (8156 records, 6 fields): 19 millis
pass1 - making usageList (24 chroms): 3 millis
pass2 - checking and writing primary data (9811 records, 6 fields): 21 millis
pass1 - making usageList (24 chroms): 3 millis
pass2 - checking and writing primary data (5452 records, 6 fields): 21 millis
pass1 - making usageList (24 chroms): 2 millis
pass2 - checking and writing primary data (2659 records, 6 fields): 13 millis
pass1 - making usageList (24 chroms): 2 millis
pass2 - checking and writing primary data (895 records, 6 fields): 7 millis
pass1 - making usageList (24 chroms): 4 millis
pass2 - checking and writing primary data (10075 records, 6 fields): 33 millis
pass1 - making usageList (25 chroms): 28 millis
pass2 - checking and writing primary data (55455 records, 6 fields): 115 millis
pass1 - making usageList (24 chroms): 3 millis
pass2 - checking and writing primary data (3576 records, 6 fields): 10 milli

pass1 - making usageList (24 chroms): 3 millis
pass2 - checking and writing primary data (9243 records, 6 fields): 24 millis
pass1 - making usageList (24 chroms): 13 millis
pass2 - checking and writing primary data (24942 records, 6 fields): 81 millis
pass1 - making usageList (23 chroms): 2 millis
pass2 - checking and writing primary data (1857 records, 6 fields): 7 millis
pass1 - making usageList (24 chroms): 3 millis
pass2 - checking and writing primary data (4513 records, 6 fields): 12 millis
pass1 - making usageList (25 chroms): 5 millis
pass2 - checking and writing primary data (9131 records, 6 fields): 30 millis
pass1 - making usageList (23 chroms): 4 millis
pass2 - checking and writing primary data (7536 records, 6 fields): 20 millis
pass1 - making usageList (24 chroms): 4 millis
pass2 - checking and writing primary data (4084 records, 6 fields): 13 millis
pass1 - making usageList (24 chroms): 2 millis
pass2 - checking and writing primary data (2277 records, 6 fields): 7 millis


pass1 - making usageList (25 chroms): 2 millis
pass2 - checking and writing primary data (6054 records, 6 fields): 8 millis
pass1 - making usageList (24 chroms): 2 millis
pass2 - checking and writing primary data (2317 records, 6 fields): 4 millis
pass1 - making usageList (24 chroms): 1 millis
pass2 - checking and writing primary data (4977 records, 6 fields): 8 millis
pass1 - making usageList (25 chroms): 3 millis
pass2 - checking and writing primary data (11960 records, 6 fields): 15 millis
pass1 - making usageList (25 chroms): 4 millis
pass2 - checking and writing primary data (21815 records, 6 fields): 24 millis
pass1 - making usageList (25 chroms): 2 millis
pass2 - checking and writing primary data (7127 records, 6 fields): 10 millis
pass1 - making usageList (24 chroms): 2 millis
pass2 - checking and writing primary data (9226 records, 6 fields): 12 millis
pass1 - making usageList (24 chroms): 3 millis
pass2 - checking and writing primary data (13532 records, 6 fields): 16 millis


pass1 - making usageList (23 chroms): 1 millis
pass2 - checking and writing primary data (1000 records, 6 fields): 4 millis
pass1 - making usageList (24 chroms): 3 millis
pass2 - checking and writing primary data (11372 records, 6 fields): 35 millis
pass1 - making usageList (24 chroms): 3 millis
pass2 - checking and writing primary data (9814 records, 6 fields): 15 millis
pass1 - making usageList (24 chroms): 4 millis
pass2 - checking and writing primary data (15584 records, 6 fields): 31 millis
pass1 - making usageList (24 chroms): 3 millis
pass2 - checking and writing primary data (12678 records, 6 fields): 19 millis
pass1 - making usageList (24 chroms): 6 millis
pass2 - checking and writing primary data (4003 records, 6 fields): 8 millis
pass1 - making usageList (25 chroms): 3 millis
pass2 - checking and writing primary data (9435 records, 6 fields): 27 millis
pass1 - making usageList (25 chroms): 2 millis
pass2 - checking and writing primary data (6761 records, 6 fields): 12 millis

In [6]:
out_file

'/projects/ps-yeolab3/bay001/maps/current_annotations/se_peak_bigbeds/689_02.basedon_689_02.peaks.l2inputnormnew.bed.compressed.bed.p3f2.bed'

# Iterate over blacklist removed peak files and conver to bigbed

In [ ]:
p = 3

all_peaks = glob.glob(os.path.join(params['peak_blacklistremoved']['peak_dir'],'*.compressed2.bed.blacklist_removed.bed'))
print(len(all_peaks))

chrom_sizes = '/projects/ps-yeolab/genomes/hg19/hg19.chrom.sizes'
progress = tnrange(len(all_peaks))

for fc in [2, 4, 8]:
    print(fc)
    for peak in all_peaks:
        out_file = os.path.join(params['peak_blacklistremoved']['output_dir'], os.path.basename(peak) + '.p{}f{}.bed'.format(p, fc))
        sorted_bed_file = out_file + '.sorted.bed'
        bb_file = sorted_bed_file + '.bb'

        filter_and_change_namefield(peak, out_file, p, fc)
        ! sort -k1,1 -k2,2n $out_file > $sorted_bed_file
        ! bedToBigBed $sorted_bed_file $chrom_sizes $bb_file
        progress.update(1)

In [5]:
# delete empty files

wd = params['peak']['output_dir']

! find $wd -type f -empty -delete

In [4]:
wd = params['peak']['output_dir']
wd

'/projects/ps-yeolab3/bay001/maps/current_annotations/se_peak_bigbeds/'